In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from random import randint
import glob
import os
import random

from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


import torchvision
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torch.optim as optim

%matplotlib inline

In [ ]:
class CamaraModelDataset(Dataset):

    def __init__(self, path2data, transforms=None, is_train = False):
        

        self.transform = transforms 
        
        self.X= []
        self.y = []
        count = 0
        labels = {}
        for subdir, dirs, files in os.walk(path):
            if subdir.split('/')[-1] != '':
                labels[count] = subdir.split('/')[-1]
                path_folder = os.path.join(subdir, '*.jpg')
                files = glob.glob(path_folder) 
                self.X.extend(files)
                self.y.extend(np.full((1, len(files)), count, dtype=int)[0])
                
                count = count + 1
                
        self.labels = labels
            
    def __getitem__(self, index):
        
        path = self.X[index]
        label = self.y[index]

        with open(path, 'rb') as f:
            flbase = os.path.basename(path)
            
            with Image.open(f) as img:
                 image = img.convert('RGB')
                    
        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def __len__(self):
        return len(self.X)

In [ ]:
class Image2Vector(object):

    def __call__(self, image):

        image = np.asarray(image)
        image = np.reshape(image, (image.shape[0]*image.shape[1], image.shape[2]))

        return image

In [ ]:
class SpatialFiltering(object):
    
    def __call__(self, image):

        image = image - image.mean()
        return image

In [ ]:
img_width = img_height=128
nb_channels = 3
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((img_width, img_height)),
        transforms.ToTensor(),
        SpatialFiltering()
    ])
}

In [ ]:
path = 'data/flowers/'

In [ ]:
batch_size = 512
n_epochs = 100
learningRate = 0.5

In [ ]:
dsets = {'train': CamaraModelDataset(path, transforms=data_transforms['train'],  is_train = True)}
dloader ={'train': torch.utils.data.DataLoader(dsets['train'], batch_size=batch_size, shuffle=True)} 

In [ ]:
labels = dsets['train'].labels
print('Labels of the dataset: {}'.format(labels))

In [ ]:
def denormalize(image, mean=imagenet_mean, std=imagenet_std):
    inp = image.transpose((1, 2, 0))  # Channel Last
    img = std * inp + mean
    return img

In [ ]:
def CamaraPhotos(dsets, labels):
    
    rand_img = random.randrange(0, len(dsets))
    img, y = dsets[rand_img]
    img = img.numpy()
    img = img.transpose((1, 2, 0))  # Channel Last
    plt.imshow(img)
    
    plt.title('Label: {}'.format(labels[int(y)]))

    plt.axis('off')

In [ ]:
plt.figure(figsize=(20,5))
for i in range(0, 3):
    plt.subplot(1,3,i+1)

    CamaraPhotos(dsets['train'], labels)

In [ ]:
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        out = self.linear(x)
        return out

In [ ]:
input_size = nb_channels*img_width*img_height
num_classes = len(labels)

model = LogisticRegression(input_size, num_classes)

In [ ]:
criterion = torch.nn.CrossEntropyLoss(size_average=True)
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)

In [ ]:
for epoch in range(n_epochs):
    for i, (images, labels) in enumerate(dloader['train']):
        images = Variable(images.view(-1, nb_channels*img_width*img_height))
        labels = Variable(labels)

        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print ('Epoch: [%d/%d], Loss: %.4f' 
           % (epoch+1, n_epochs, loss.data[0]))

### Evaluate the model

In [ ]:
plt.figure(figsize=(20,4))
for plotIndex, badIndex in enumerate(misclassifiedIndexes[0:5]):
    plt.subplot(1, 5, plotIndex + 1)
    plt.imshow(np.reshape(X_test[badIndex], (20,20)), cmap=plt.cm.gray)
    plt.axis('off')
    plt.title('Predicted: {}, Actual: {}'.format(predictions[badIndex], y_test[badIndex]), fontsize = 15)